<h2> compare two model result </h2>


<h3>1. Load both models</h3>

In [ ]:
import os
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MODEL_PATH1 = ""
MODEL_PATH2 = ""
N_CLASSES = 12
REAL_TEST_DIR = ""
BATCH_SIZE = 24

In [ ]:
# Load model 1
model1 = models.resnet18(pretrained=False)
model1.fc = nn.Linear(model.fc.in_features, len(classes))
model1.load_state_dict(torch.load(MODEL_PATH1, map_location=DEVICE))
model1.eval()

In [ ]:
# Load model 1
model2 = models.resnet18(pretrained=False)
model2.fc = nn.Linear(model.fc.in_features, len(classes))
model2.load_state_dict(torch.load(MODEL_PATH2, map_location=DEVICE))
model2.eval()

<h3> 2. Create Accuracy Testing  and t-SNE for both model </h3>

# data Loader
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_dataset = datasets.ImageFolder(REAL_TEST_DIR, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

classes = test_dataset.classes
print(f'Classes: {classes}')
print(f'Number of test samples: {len(test_dataset)}')

In [ ]:
# Prediction using model 1
all_labels = []
all_preds = []
all_feature = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        outputs = model1(inputs)
        _, preds = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

all_labels = np.array(all_labels)
all_preds = np.array(all_preds)

# Overall Accuracy
accuracy = np.mean(all_preds == all_labels)
print(f'Baseline Model Accuracy on Real Test Set: {accuracy:.4f}')

# Classification Report
report = classification_report(all_labels, all_preds, target_names=classes)
print('Classification Report:\n', report)

In [ ]:
# Prediction using model 2
all_labels = []
all_preds = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        outputs = model2(inputs)
        _, preds = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

all_labels = np.array(all_labels)
all_preds = np.array(all_preds)

# Overall Accuracy
accuracy = np.mean(all_preds == all_labels)
print(f'Baseline Model Accuracy on Real Test Set: {accuracy:.4f}')

# Classification Report
report = classification_report(all_labels, all_preds, target_names=classes)
print('Classification Report:\n', report)

<h3> Compare the feature between model </h3>